# CTAS in Telecom Domain: Call Logs Example

In this notebook, we will work with telecom call logs and perform various operations using CTAS (Create Table As Select), transformations, constraints, and cloning.

### Step 1: Create Schema

In [ ]:

# Create a schema for telecom data
spark.sql("""CREATE SCHEMA IF NOT EXISTS telecom_ctas;""")
    

### Step 2: Create a Managed Table with CTAS

In [ ]:

# Create a managed table using the telecom call logs
spark.sql("""
CREATE OR REPLACE TABLE telecom_ctas.call_logs AS
SELECT * FROM csv.`/mnt/data/telecom_call_logs.csv`;
""")
    

### Step 3: Rename and Filter Columns

In [ ]:

# Rename and filter columns while creating a new table
spark.sql("""
CREATE OR REPLACE TABLE telecom_ctas.filtered_logs AS
SELECT Call_ID AS id, Call_Duration AS duration, Call_Time AS call_time
FROM telecom_ctas.call_logs;
""")
    

### Step 4: Add Generated Columns

In [ ]:

# Create a table with a generated column that calculates the call date
spark.sql("""
CREATE OR REPLACE TABLE telecom_ctas.generated_logs (
  id INT,
  duration INT,
  call_time TIMESTAMP,
  call_date DATE GENERATED ALWAYS AS (CAST(call_time AS DATE)) COMMENT 'Generated from call_time'
);
""")
    

### Step 5: Add Constraints

In [ ]:

# Add a CHECK constraint to ensure that the call_date is greater than a specific date
spark.sql("""
ALTER TABLE telecom_ctas.generated_logs ADD CONSTRAINT valid_date CHECK (call_date > '2020-01-01');
""")
    

### Step 6: Enrich Table with Metadata

In [ ]:

# Create a table with additional metadata and partitioning by call_date
spark.sql("""
CREATE OR REPLACE TABLE telecom_ctas.enriched_logs
COMMENT 'Enriched table with partitioning and metadata'
PARTITIONED BY (call_date)
AS SELECT id, duration, call_time, call_date FROM telecom_ctas.generated_logs;
""")
    

### Step 7: Clone the Table

In [ ]:

# Create a shallow clone of the enriched_logs table
spark.sql("""
CREATE OR REPLACE TABLE telecom_ctas.enriched_logs_clone SHALLOW CLONE telecom_ctas.enriched_logs;
""")
    

### Step 8: Cleanup

In [ ]:

# Cleanup: Drop the schema and all tables within it
spark.sql("""DROP SCHEMA telecom_ctas CASCADE;""")
    